<a href="https://colab.research.google.com/github/SusmitWani/NNFL-Course-Fall-2020/blob/master/NNFL_Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing required libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from collections import Counter
np.random.seed(42)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


#Importing and cleaning the dataframe.


In [2]:
df = pd.read_csv('/content/drive/My Drive/Datasets for ML DL/BankData/ChurnBank.csv')

In [3]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
df.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,49179.075167,4.701315e+04,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,71173.768783,6.934939e+04,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,2984.750000,2.666250e+03,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,21200.000000,2.008850e+04,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,64006.250000,6.016475e+04,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


#####Check if there are any missing entries or object data


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

Divide code into train and test dataframes for training and validation

In [6]:
X = df[df.columns[1:-1].values]
y = df[df.columns[-1]]
scalar = StandardScaler()
X = scalar.fit_transform(X.values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3333, stratify = y, random_state=42)

##Some functions that will be useful to us later in the notebook


In [7]:
def sigmoid(n):
  return 1/(1+(np.exp(-n)))

In [8]:
#Function will initialize W and b vectors as random number vectors of required dimensions.
def initialize_rand(X):
    num_cols = X.shape[1]
    num_data = X.shape[0]

    b = 1
    W = np.zeros([num_cols, 1])
    return W, b, num_data, num_cols

In [9]:
#Function which validates the obtained W and b values on the validation set

def validation(X_test, y_test, W_final, b_final):
    num_data = X_test.shape[0]
    correct_pred = 0
    y_pred = []
    for i in range(num_data):
        temp = np.sum(np.dot(W_final.T, X_test[i])[0] + b_final)
        if temp>=0:
            y_pred.append(1);
        else:
            y_pred.append(0);
        if y_pred[i]==y_test.iloc[i]:
            correct_pred = correct_pred+1
    accuracy = correct_pred/num_data
    return accuracy, y_pred

#Code to actually train the perceptron

In [10]:
#Function where actual learning takes place by the Unified Learning Rule

def train_ULR(X_train, y_train, W, b):
    for i in range(X_train.shape[0]):
        temp = np.sum(np.dot(W.T, X_train[i])[0] + b)
        if temp>=0:
            y_pred = 1
        else:
            y_pred = 0
        e = y_train.iloc[i] - y_pred
        W = W + e*X_train[i].reshape(23,1)
        b = b + e
    return W, b

In [11]:
#Function to repeat train_ULR function

def repeat_train(W, b, X_train, y_train, num_iter = 100):
    for i in range(num_iter):
        W, b = train_ULR(X_train, y_train, W, b)
    return W, b

In [12]:
W, b, num_data, num_cols = initialize_rand(X_train)
W_final, b_final = repeat_train(W, b, X_train, y_train, 10)
accuracy, y_pred1 = validation(X_test, y_test, W_final, b_final)
cm1 = confusion_matrix(y_test, y_pred1)
print(accuracy)
print(cm1)
# print(W_final)
# print(b_final)

0.768076807680768
[[6990  797]
 [1522  690]]


#Using Gradient Descent.
####*Loss function implemented is binary cross entropy*

In [13]:
def forward_prop(X, y, W, b):
    num_data = X.shape[1]
    z = sigmoid(np.dot(W.T, X.T) + b)
    #The defined cost function is for binary cross entropy.
    cost = np.squeeze((np.sum((np.array(y))*np.log(z)-(1-np.array(y))*np.log(1-z))))

    dw = (1/num_data)*np.dot(X.T, (z-np.array(y)).T)
    db = (1/num_data)*(np.sum(z-np.array(y)))
    return cost, dw, db

In [14]:
def train_SGD(X_train, y_train, num_iter, lr=0.0002):
    W, b, num_data, num_cols = initialize_rand(X_train)
    
    costs = []
    for i in range(num_iter):
        cost, dw, db = forward_prop(X_train, y_train, W, b)
        W = W - lr*dw
        b = b - lr*db

        if i%50==0:
            costs.append(cost)
        if i==num_iter-1:
            if db>0.001:
                print("Didn't converge. Increase lr or increase num_iter")
    return W, b, costs


In [15]:
W_final, b_final, costs = train_SGD(X_train, y_train, 5000)
acc, y_pred2 = validation(X_test, y_test, W_final, b_final)
cm2 = confusion_matrix(y_test, y_pred2)
print(acc)
print(cm2)

0.8079807980798079
[[7550  237]
 [1683  529]]


#Using normal Logistic Regression function
#####Used as a measure of how good the model is. And as we can see, its not a great model.





In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
regressor = LogisticRegression(random_state=42)
regressor.fit(X_train, y_train)
y_pred3 = regressor.predict(X_test)
accuracy_score(y_test, y_pred3)

0.8079807980798079

#Bayes Rule

For a given row in x_test, find probabilities of y being 0 or 1 using above formula.
So what needs to be done is as follows:

*   For an assumed y, find the probability of all the X values in X_test from X_train by sampling whole columns.
*   Take product of these and multiply it with the probability of the assumed y value occuring
*   We do not need to divide the above product with the P(sigma(X)) as it is going to be common in both cases.

Because the data is continuous data, we will be using the Gaussian Distribution to calculate probabilities



In [17]:
X = df[df.columns[1:-1].values]
y = df[df.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33333, stratify = y, random_state=42)

In [18]:
#Calculate probability of different Y's

def probab_y(y_train):
    y_dic = {};
    var = np.unique(y_train, return_counts=True)
    for y in range(len(var[0])):
        y_dic[var[0][y]] = var[1][y]/len(y_train)
    return y_dic

In [19]:
# y_dic = probab_y(y_train)
# print(sum(y_dic.values()))
# print(y_dic.keys())
# print(y_dic.values())
# y_dic

In [20]:
#Get separate dataframes for each y value and return a dictionary.

def get_separate_df(X_train, y_train):
    y_dfs = {}
    var = np.unique(y_train)
    for y in var:
        inc_index = np.where(y_train==y)[0]
        y_dfs[y] = (X_train.iloc[inc_index])
    return y_dfs

In [21]:
# y_dfs = get_separate_df(X_train, y_train)
# print(y_dfs.keys())

In [22]:
#This function will get us the mean and std dev of X variables

def get_dist(X_train, y_train):
    y_dfs = get_separate_df(X_train, y_train)
    y_mean = {}
    y_std = {}
    for y in y_dfs.keys():
        y_mean[y] = np.mean(y_dfs[y])
        y_std[y] = np.std(y_dfs[y])
    return y_mean, y_std

In [23]:
# y_mean, y_std = get_dist(X_train, y_train)
# print(y_mean[0].values)
# print(y_mean[0])

In [24]:
#Calculate probabilities whenever needed using this function

def calc_probab(mean, dev, x):
    pow = np.exp(-1*((x-mean)**2)/(2*dev*dev))
    probab = pow/(np.sqrt(2*np.pi*dev*dev))
    return probab

####We are done with all the helper functions. Now for the actual implementation

In [25]:
def predict_with_Bayes(X_train, y_train, X_test, y_test):
    priors_dic = probab_y(y_train)
    y_mean, y_std = get_dist(X_train, y_train)
    y_pred = []
    #iterate thru each example
    for i in range(len(y_test)):
        y_temp = []
        for y in priors_dic.keys():
            prob = np.prod(calc_probab(y_mean[y], y_std[y], X_test.iloc[i].values).values)*priors_dic[y]
            y_temp.append(prob)
        max_y = np.argmax(y_temp)
        keys = list(priors_dic)
        y_pred.append(keys[max_y])
    return y_pred

In [26]:
y_predNB = predict_with_Bayes(X_train, y_train, X_test, y_test)
cmNB = confusion_matrix(y_test, y_predNB)

In [27]:
print(accuracy_score(y_test, y_predNB))
print(cmNB)

0.7485
[[6216 1572]
 [ 943 1269]]
